In [1]:
import csv
import codecs
import fuzzywuzzy
import re
from itertools import groupby
from fuzzywuzzy import process, fuzz
from collections import namedtuple
import requests
import pandas as pd
import numpy as np
import os
import lxml

C:\Users\10nikov\Anaconda3\envs\myenv\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
%run shared.py

In [3]:
def read_csv(file):
    with codecs.open(file, encoding='utf-8') as f:
        data = list(csv.reader(f, delimiter=';', quotechar='"'))[1:]
        return data

In [4]:
catalog = [c for c in read_csv('catalog.txt') if len(c) >= 6 and c[5] == "ru"]
len(catalog)

366289

In [5]:
liter = read_csv('liter.csv')
len(liter)

543

In [6]:
curr_author = None
for l in liter:
    if l[0]:
        curr_author = l[0]
    else:
        l[0] = curr_author

In [7]:
def get_surname(s):
    return re.split(',? ', s)[0]


In [8]:
catalog_by_author_surname = dict((surname, list(entries)) for surname, entries in groupby(catalog, lambda x: x[0]))


In [9]:
for surname, books in catalog_by_author_surname.items():
    for b in books:
        b[3] = re.sub(r"(\s?[А-ЯЁ]\.\s?){,2}" + re.escape(surname) + r"(\s?[А-ЯЁ]\.\s?){,2}", " ", b[3])

In [10]:
polka_pairs = [(t, a) for t,a in [["Герой нашего времени","Михаил Лермонтов"],["Анна Каренина","Лев Толстой"],["Мёртвые души","Николай Гоголь"],["Капитанская дочка","Александр Пушкин"],["Война и мир","Лев Толстой"],["Котлован","Андрей Платонов"],["Москва — Петушки","Венедикт Ерофеев"],["Братья Карамазовы","Фёдор Достоевский"],["Идиот","Фёдор Достоевский"],["Петербург","Андрей Белый"],["Преступление и наказание","Фёдор Достоевский"],["Вишнёвый сад","Антон Чехов"],["Конармия","Исаак Бабель"],["Колымские рассказы","Варлам Шаламов"],["Мелкий бес","Фёдор Сологуб"],["Зависть","Юрий Олеша"],["Бесы","Фёдор Достоевский"],["Евгений Онегин","Александр Пушкин"],["Шинель","Николай Гоголь"],["Козлиная песнь","Константин Вагинов"],["Дар","Владимир Набоков"],["Чевенгур","Андрей Платонов"],["Школа для дураков","Саша Соколов"],["Повести Белкина","Александр Пушкин"],["Обломов","Иван Гончаров"],["Мы","Евгений Замятин"],["Ревизор","Николай Гоголь"],["Мастер и Маргарита","Михаил Булгаков"],["Один день Ивана Денисовича","Александр Солженицын"],["Отцы и дети","Иван Тургенев"],["Горе от ума","Александр Грибоедов"],["Нос","Николай Гоголь"],["Двенадцать","Александр Блок"],["Борис Годунов","Александр Пушкин"],["История одного города","Михаил Салтыков-Щедрин"],["Двенадцать стульев","Илья Ильф"],["Приглашение на казнь","Владимир Набоков"],["Доктор Живаго","Борис Пастернак"],["Белая гвардия","Михаил Булгаков"],["Хаджи-Мурат","Лев Толстой"],["Былое и думы","Александр Герцен"],["Чайка","Антон Чехов"],["Смерть Ивана Ильича","Лев Толстой"],["Медный всадник","Александр Пушкин"],["Записки сумасшедшего","Николай Гоголь"],["Маленькие трагедии","Александр Пушкин"],["Соборяне","Николай Лесков"],["Четвёртая проза","Осип Мандельштам"],["Бедная Лиза","Николай Карамзин"],["Записки охотника","Иван Тургенев"],["Старуха","Даниил Хармс"],["Пушкинский дом","Андрей Битов"],["Защита Лужина","Владимир Набоков"],["Жизнь и судьба","Василий Гроссман"],["Господа Головлёвы","Михаил Салтыков-Щедрин"],["Тёмные аллеи","Иван Бунин"],["Житие протопопа Аввакума","Аввакум Петров"],["Невский проспект","Николай Гоголь"],["Левша","Николай Лесков"],["Жизнь Арсеньева","Иван Бунин"],["На дне","Максим Горький"],["Поэма без героя","Анна Ахматова"],["Василий Тёркин","Александр Твардовский"],["Сандро из Чегема","Фазиль Искандер"],["Дом на набережной","Юрий Трифонов"],["Очарованный странник","Николай Лесков"],["Степь","Антон Чехов"],["Золотой телёнок","Евгений Петров"],["Архипелаг ГУЛАГ","Александр Солженицын"],["Факультет ненужных вещей","Юрий Домбровский"],["Пиковая дама","Александр Пушкин"],["Портрет","Николай Гоголь"],["Демон","Михаил Лермонтов"],["Дама с собачкой","Антон Чехов"],["Три сестры","Антон Чехов"],["Леди Макбет Мценского уезда","Николай Лесков"],["Одесские рассказы","Исаак Бабель"],["Лолита","Владимир Набоков"],["Записки из подполья","Фёдор Достоевский"],["Призрак Александра Вольфа","Гайто Газданов"],["Слово о полку Игореве","в спискеДревняя Русь"],["Норма","Владимир Сорокин"],["Картины прошедшего","Александр Сухово-Кобылин"],["Дворянское гнездо","Иван Тургенев"],["Тихий Дон","Михаил Шолохов"],["Город Эн","Леонид Добычин"],["Старосветские помещики","Николай Гоголь"],["Облако в штанах","Владимир Маяковский"],["Случаи","Даниил Хармс"],["Прощание с Матёрой","Валентин Распутин"],["Заповедник","Сергей Довлатов"],["Детство. Отрочество. Юность","Лев Толстой"],["Севастопольские рассказы","Лев Толстой"],["Бедные люди","Фёдор Достоевский"],["Кому на Руси жить хорошо","Николай Некрасов"],["Зангези","Велимир Хлебников"],["Шум времени","Осип Мандельштам"],["Голубая книга","Михаил Зощенко"],["Чапаев и Пустота","Виктор Пелевин"],["Вечера на хуторе близ Диканьки","Николай Гоголь"],["Путешествие из Петербурга в Москву","Александр Радищев"],["Цыганы","Александр Пушкин"],["В овраге","Антон Чехов"],["Опавшие листья","Василий Розанов"],["Ёлка у Ивановых","Александр Введенский"],["Во сне ты горько плакал","Юрий Казаков"],["Поэма Горы","Марина Цветаева"],["Время ночь","Людмила Петрушевская"]]]


In [11]:
def match_catalog_entry(surname, book):
    cat_by_author = catalog_by_author_surname.get(surname)
    if not cat_by_author:
        print("strange surname " + surname)
        return []
        
    cat_book_names = [x[3] for x in cat_by_author]
    best_book, score = process.extractOne(book, cat_book_names, scorer=fuzz.token_sort_ratio)
    if score <= 90:
        return []
    
    best_cat_entry = next(x for x in cat_by_author if x[3] == best_book)
    return best_cat_entry

cat_columns = ["cat_f", "cat_i", "cat_o", "cat_title", "cat_comment", "cat_lang", "cat_issue_year", "cat_collection", "cat_id"]
matched_with_mukhin = pd.DataFrame([match_catalog_entry(get_surname(fio), book) for fio, wiki1, wiki2, book, *rest in liter], columns=cat_columns)
matched_with_polka = pd.DataFrame([match_catalog_entry(fi.split(" ")[1], book) for book, fi in polka_pairs], columns=cat_columns)

strange surname спискеДревняя


In [12]:
mukhin_liter = pd.DataFrame(liter, columns=["mukhin_fio", "mukhin_wiki1", "mukhin_libru", "mukhin_title", "mukhin_year", "mukhin_wiki2"])
with_mukhin = pd.concat([matched_with_mukhin, mukhin_liter], axis=1)
#with_mukhin.cat_id = with_mukhin.cat_id.astype("str")
unmatched_mukhin = with_mukhin[with_mukhin.cat_id.isnull()]
with_mukhin = with_mukhin[with_mukhin.cat_id.notnull()]

with_polka = pd.concat([matched_with_polka, pd.DataFrame(polka_pairs, columns=["polka_title", "polka_fi"])], axis=1)
#with_polka.cat_id = with_polka.cat_id.astype("str")
unmatched_polka = with_polka[with_polka.cat_id.isnull()]
with_polka = with_polka[with_polka.cat_id.notnull()]


In [13]:
manually_selected_cat_ids = set(map(str, [47023, 186062, 53076, 336019, 143082, 402759, 146717, 102649, 18565, 168592, 93198]))
manually_selected_cat = pd.DataFrame([c for c in catalog if c[-1] in manually_selected_cat_ids], columns=cat_columns)
manually_selected_cat

,cat_f,cat_i,cat_o,cat_title,cat_comment,cat_lang,cat_issue_year,cat_collection,cat_id
0,Белый,Андрей,,Серебряный голубь,,ru,1909,,93198
1,Вагинов,Константин,Константинович,Бамбочада,,ru,2008,,168592
2,Газданов,Гайто,Иванович,Пробуждение,,ru,,,18565
3,Герцен,Александр,Иванович,Кто виноват?,,ru,1948,,102649
4,Казаков,Юрий,Павлович,Голубое и зеленое,,ru,1963,,146717
5,Карамзин,Николай,Михайлович,Рыцарь нашего времени,,ru,1803,,402759
6,Лермонтов,Михаил,Юрьевич,Княгиня Лиговская,,ru,,Проза Михаила Лермонтова 0,143082
7,Радищев,Александр,Николаевич,Дневник одной недели,,ru,,,336019
8,Розанов,Василий,Васильевич,Черный огонь,,ru,,,47023
9,Соколов,Саша,,Палисандрия,,ru,,,53076


In [94]:
def get_best_polka_book(author_fio, title):
    polka_of_author = [(name, fi)
                              for name, fi in polka_pairs 
                              if author_fio.find(fi.split(" ")[1]) >= 0]
    polka_titles = [name for name, fi in polka_of_author]
    #title = manual_mukhin_to_polka_correspondences.get(title) or title
    extracted = process.extractOne(title, polka_titles, scorer=fuzz.token_sort_ratio) or (None, 0)
    best_book, score = extracted
    if score > 90:
        return list(next(filter(lambda p: p[0] == best_book, polka_of_author)))
    return [None, None]

unmatched_mukhin_with_polkas = pd.DataFrame(
    [[fio, title] + get_best_polka_book(fio, title) for fio, title in unmatched_mukhin[["mukhin_fio", "mukhin_title"]].values], 
    columns=["mukhin_fio", "mukhin_title", "polka_title", "polka_fi"])
unmatched_polka_not_mentioned_in_mukhin = pd.DataFrame([[title, fi] for title, fi 
                                           in unmatched_polka[["polka_title", "polka_fi"]].values 
                                           if len(unmatched_mukhin_with_polkas[
                                               (unmatched_mukhin_with_polkas.polka_fi == fi) 
                                               & (unmatched_mukhin_with_polkas.polka_title == title)
                                           ]) == 0
                                          ], columns=["polka_title", "polka_fi"])

joined = with_mukhin.set_index(cat_columns).join(with_polka.set_index(cat_columns), how="outer").reset_index()
joined.loc[joined.cat_title == "Преступление и наказание", "cat_id"] = '261797'
joined.loc[joined.cat_title == "Анна Каренина", "cat_id"] = '416697'
joined.loc[joined.polka_title == "Петербург"]
joined.loc[joined.polka_title == "Горе от ума", "cat_id"] = '397134'
joined.loc[joined.polka_title == "Детство. Отрочество. Юность", "cat_id"] = '535985'
joined.loc[joined.polka_title == "Капитанская дочка", "cat_id"] = '276903'
joined.loc[joined.polka_title == "Обломов", "cat_id"] = '485060'
joined.loc[joined.polka_title == "Отцы и дети", "cat_id"] = '499439'
joined.loc[joined.polka_title == "Чапаев и пустота", "cat_id"] = '42053'
joined.loc[joined.polka_title == "Бедные люди", "cat_id"] = '492140'
joined.loc[joined.polka_title == "Картины прошедшего", "cat_id"] = '147229'

joined.loc[((joined.cat_id == '105610') & joined.polka_title.isnull()), ["polka_title", "polka_fi"]] = ["Двенадцать стульев","Илья Ильф"]
joined = joined.drop(joined[(joined.cat_id == '105610') & (joined.mukhin_wiki1.isnull())].index)

all_lists = pd.concat([joined, unmatched_mukhin_with_polkas, unmatched_polka_not_mentioned_in_mukhin, manually_selected_cat]).reset_index()
all_lists.loc[all_lists.polka_title == "Житие протопопа Аввакума", "cat_id"] = '472658'
all_lists.loc[all_lists.polka_title == "Слово о полку Игореве", "cat_id"] = '150566'
all_lists.loc[all_lists.polka_title == "Опавшие листья", "cat_id"] = '175663'
all_lists.loc[all_lists.polka_title == "Ёлка у Ивановых", "cat_id"] = 'elka'

In [95]:
for c in ["mukhin_fio", "cat_f", "cat_i", "cat_o", "polka_fi"]:
    all_lists[c] = all_lists[c].str.replace(",", "").str.replace("ё", "е")
all_lists["some_f_polka"] = np.where(
    all_lists.cat_f.isnull() & all_lists.polka_fi.notnull(), 
    all_lists.polka_fi.str.split(" ").str.get(-1) + " " + all_lists.polka_fi.str.split(" ").str.get(0), 
    all_lists.cat_f + " " + all_lists.cat_i)
all_lists["fi"] = np.where(
    all_lists.some_f_polka.isnull() & all_lists.mukhin_fio.notnull(), 
    all_lists.mukhin_fio.str.split(" ").str.get(0) + " " + all_lists.mukhin_fio.str.split(" ").str.get(1),
    all_lists.some_f_polka)
all_lists = all_lists.drop("some_f_polka", axis=1)
for_grouping = all_lists
set(all_lists.fi)

{'Абрамов Федор',
 'Аверченко Аркадий',
 'Айтматов Чингиз',
 'Аксенов Василий',
 'Акунин Борис',
 'Алексиевич Светлана',
 'Алексин Анатолий',
 'Алешковский Петр',
 'Алешковский Юз',
 'Андреев Даниил',
 'Андреев Леонид',
 'Арсеньев Владимир',
 'Астафьев Виктор',
 'Ахматова Анна',
 'Бабель Исаак',
 'Бажов Павел',
 'Белов Василий',
 'Белый Андрей',
 'Беляев Александр',
 'Бестужев-Марлинский Александр',
 'Битов Андрей',
 'Блок Александр',
 'Бондарев Юрий',
 'Булгаков Михаил',
 'Булычев Кир',
 'Бунин Иван',
 'Быков Василь',
 'Быков Дмитрий',
 'Вагинов Константин',
 'Вайнер Аркадий',
 'Васильев Борис',
 'Введенский Александр',
 'Веллер Михаил',
 'Водолазкин Евгений',
 'Войнович Владимир',
 'Воробьев Константин',
 'Газданов Гайто',
 'Гайдар Аркадий',
 'Гарин-Михайловский Николай',
 'Гаршин Всеволод',
 'Геласимов Андрей',
 'Герцен Александр',
 'Гладков Федор',
 'Гоголь Николай',
 'Гончаров Иван',
 'Горький Максим',
 'Гранин Даниил',
 'Грибоедов Александр',
 'Грин Александр',
 'Гришковец Евгени

In [96]:
all_lists.loc[(all_lists.cat_id == '271485') | (all_lists.fi == "Ильф Илья") | (all_lists.fi == "Петров Евгений"), "fi"] = "Ильф и Петров"
all_lists[all_lists.cat_id == '271485']

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
307,308,Остап Бендер 2,полная версия,Петров,Евгений,271485,1994,ru,Петрович,Золотой Теленок,Петров Евгений Ильф Илья,,Золотой теленок,,https://ru.wikipedia.org/wiki/%D0%97%D0%BE%D0%...,1931,Евгений Петров,Золотой телёнок,Ильф и Петров


In [71]:
joined

,cat_f,cat_i,cat_o,cat_title,cat_comment,cat_lang,cat_issue_year,cat_collection,cat_id,mukhin_fio,mukhin_wiki1,mukhin_libru,mukhin_title,mukhin_year,mukhin_wiki2,polka_title,polka_fi
0,Абрамов,Фёдор,Александрович,Братья и сестры,,ru,1982,Пряслины 1,158030,"Абрамов, Федор Александрович",https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...,http://lib.ru/PROZA/ABRAMOW/,Братья и сестры,1958,https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%...,NaN,NaN
1,Абрамов,Фёдор,Александрович,Две зимы и три лета,,ru,1969,Пряслины 2,158033,"Абрамов, Федор Александрович",,,Две зимы и три лета,1968,,NaN,NaN
2,Абрамов,Фёдор,Александрович,Дом,,ru,1984,Пряслины 4,158032,"Абрамов, Федор Александрович",,,Дом,1978,,NaN,NaN
3,Абрамов,Фёдор,Александрович,Пути-перепутья,,ru,1985,Пряслины 3,158036,"Абрамов, Федор Александрович",,,Пути-перепутья,1973,,NaN,NaN
4,Аверченко,Аркадий,Тимофеевич,Подходцев и двое других,,ru,1917,,159231,"Аверченко, Аркадий Тимофеевич",https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...,http://lib.ru/RUSSLIT/AWERCHENKO/,Подходцев и двое других,1917,,NaN,NaN
5,Аверченко,Аркадий,Тимофеевич,Разумная экономия,,ru,,,417321,"Аверченко, Аркадий Тимофеевич",,,Разумная экономия,ок.1912,,NaN,NaN
6,Аверченко,Аркадий,Тимофеевич,Шутка мецената,,ru,2001,,159241,"Аверченко, Аркадий Тимофеевич",,,Шутка мецената,1925,https://ru.wikipedia.org/wiki/%D0%A8%D1%83%D1%...,NaN,NaN
7,Айтматов,Чингиз,,Белый пароход,,ru,1983,,150859,"Айтматов, Чингиз Торекулович",https://ru.wikipedia.org/wiki/%D0%90%D0%B9%D1%...,http://lib.ru/PROZA/AJTMATOW/,Белый пароход,1970,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,NaN,NaN
8,Айтматов,Чингиз,,"Пегий пес, бегущий краем моря",,ru,,,65797,"Айтматов, Чингиз Торекулович",,,"Пегий пес, бегущий краем моря",1977,https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D0%...,NaN,NaN
9,Айтматов,Чингиз,,Плаха,,ru,,,65796,"Айтматов, Чингиз Торекулович",,,Плаха,1986,https://ru.wikipedia.org/wiki/%D0%9F%D0%BB%D0%...,NaN,NaN


In [18]:
import os

def list_dir_with_ext(path, ext=''):
    return [path + '/' + f for f in os.listdir(path) if f.endswith(ext)]

books = list_dir_with_ext('fb2', 'fb2.zip')
len(books)

451

In [19]:
import zipfile
for b in progressify_list(list_dir_with_ext('fb2', ext='fb2.zip')):
    print(b)
    zip_ref = zipfile.ZipFile(b, 'r')
    zip_ref.extractall('fb2.unzipped')
    zip_ref.close()

fb2/Abramov_Pryasliny_1_Bratya-i-sestry.edbdyw.158030.fb2.zip
fb2/Abramov_Pryasliny_2_Dve-zimy-i-tri-leta.hU9IrQ.158033.fb2.zip
fb2/Abramov_Pryasliny_3_Puti-pereputya.ktaB0Q.158036.fb2.zip
fb2/Abramov_Pryasliny_4_Dom.FWiheg.158032.fb2.zip
fb2/Ahmatova_Poema-bez-geroya.J3oj-w.134319.fb2.zip
fb2/Akunin_Priklyucheniya-Erasta-Fandorina_12_Nefritovye-chetki.-trNMQ.144729.fb2.zip
fb2/Akunin_Priklyucheniya-Erasta-Fandorina_2_Tureckiy-gambit.xK5KVg.305713.fb2.zip
fb2/Akunin_Priklyucheniya-Erasta-Fandorina_7_Statskiy-sovetnik.5irVkw.461230.fb2.zip
fb2/Akunin_Priklyucheniya-Nikolasa-Fandorina_1_Altyn-Tolobas.Uz6tpg.131819.fb2.zip
fb2/Akunin_Priklyucheniya-Nikolasa-Fandorina_3_F-M-.cmyzNg.191882.fb2.zip
fb2/Aleksievich_Golosa-Utopii_1_U-voyny-ne-zhenskoe-lico-.VNQLMg.70938.fb2.zip
fb2/Aleksievich_Golosa-Utopii_3_Cinkovye-malchiki.NVa1uw.427115.fb2.zip
fb2/Aleksievich_Golosa-Utopii_5_Vremya-sekond-hend.9FJziw.426019.fb2.zip
fb2/Aleksin_Bezumnaya-Evdokiya.hFi8Yw.347944.fb2.zip
fb2/Aleksin_Moy-brat-

fb2/Gogol_Mertvye-dushi.b_xm6g.102645.fb2.zip
fb2/Gogol_Mirgorod_1_Starosvetskie-pomeshchiki.Kzi3zQ.159963.fb2.zip
fb2/Gogol_Mirgorod_3_Viy.JV-38g.74285.fb2.zip
fb2/Gogol_Nos.yMKAWA.96412.fb2.zip
fb2/Gogol_Portret.3kTa3Q.96414.fb2.zip
fb2/Gogol_Revizor.9agRfg.74281.fb2.zip
fb2/Gogol_Shinel.JPeMaQ.281553.fb2.zip
fb2/Gogol_Vechera-na-hutore-bliz-Dikanki.jldFuA.74286.fb2.zip
fb2/Gogol_Zapiski-sumasshedshego.xv4HqQ.96420.fb2.zip
fb2/Goncharov_Obryv.pyXIjw.74105.fb2.zip
fb2/Goncharov_Obyknovennaya-istoriya.fyJm1A.173116.fb2.zip
fb2/Gorkiy_Delo-Artamonovyh.O2tuGQ.225353.fb2.zip
fb2/Gorkiy_Mat.bDeL0g.19772.fb2.zip
fb2/Gorkiy_Na-dne.oTygrQ.19794.fb2.zip
fb2/Gorkiy_Trilogiya-Maksim-Gorkiy-_3_Moi-universitety.A4iIZw.19780.fb2.zip
fb2/Granin_Eshche-zameten-sled.qk5PRw.149025.fb2.zip
fb2/Granin_Idu-na-grozu.rC7RHg.499588.fb2.zip
fb2/Granin_Iskateli.bhevgw.498584.fb2.zip
fb2/Granin_Kartina.pp67Hg.20477.fb2.zip
fb2/Granin_Nash-dorogoy-Roman-Avdeevich.mi15NA.490942.fb2.zip
fb2/Grin_Alye-parusa.LSYj6A

fb2/Pomyalovskiy_Molotov.azA9hw.320174.fb2.zip
fb2/Pomyalovskiy_Ocherki-bursy.N9mZXA.313626.fb2.zip
fb2/Popov_Inogda-promelknet.bC_6Gw.311032.fb2.zip
fb2/Prilepin_Obitel.sxveOQ.365649.fb2.zip
fb2/Prilepin_Patologii.GwiXKw.238059.fb2.zip
fb2/Prilepin_Sankya.rsodcw.239744.fb2.zip
fb2/Prishvin_Kashcheeva-cep.N74dGw.182848.fb2.zip
fb2/Pristavkin_Gorodok.92VeiA.445820.fb2.zip
fb2/Pristavkin_Kukushata-ili-Zhalobnaya-pesn-dlya-uspokoeniya-serdca._9Mz5Q.412700.fb2.zip
fb2/Pristavkin_Nochevala-tuchka-zolotaya.CN8Maw.232269.fb2.zip
fb2/Pristavkin_Soldat-i-malchik.5Y_pgw.77143.fb2.zip
fb2/Prohanov_Gospodin-Geksogen.nkwiOg.44510.fb2.zip
fb2/Prohanov_Mesto-deystviya.5SybzQ.185965.fb2.zip
fb2/Prohanov_Shestsot-let-posle-bitvy.hZezLw.234281.fb2.zip
fb2/Prokofeva_Ostrov-kapitanov.6caUVQ.183317.fb2.zip
fb2/Prokofeva_Priklyucheniya-zhyoltogo-chemodanchika.Gn5HCQ.183209.fb2.zip
fb2/Prokofeva_Uchenik-volshebnika.9DYzFQ.183311.fb2.zip
fb2/Pushkin_Boris-Godunov.V3L2EQ.173154.fb2.zip
fb2/Pushkin_Cygany.UpTBd

In [20]:
import lxml.etree as ET

os.makedirs("txt", exist_ok=True)

fb2_to_txt_xslt = ET.parse("FB2_2_txt.xsl")
for xml_filename in progressify_list(list_dir_with_ext('fb2.unzipped', ext='fb2')):
    print(xml_filename)
    dom = ET.parse(xml_filename)
    transform = ET.XSLT(fb2_to_txt_xslt)
    txt = transform(dom)
    name = os.path.basename(xml_filename)
    with open("txt/{}.txt".format(name), "w", encoding="utf-8") as txt_file:
        txt_file.write(str(txt))

fb2.unzipped/100333.fb2
fb2.unzipped/101185.fb2
fb2.unzipped/102112.fb2
fb2.unzipped/102645.fb2
fb2.unzipped/102649.fb2
fb2.unzipped/103387.fb2
fb2.unzipped/103588.fb2
fb2.unzipped/10547.fb2
fb2.unzipped/105610.fb2
fb2.unzipped/105764.fb2
fb2.unzipped/1061.fb2
fb2.unzipped/1068.fb2
fb2.unzipped/1070.fb2
fb2.unzipped/107678.fb2
fb2.unzipped/107708.fb2
fb2.unzipped/109209.fb2
fb2.unzipped/110929.fb2
fb2.unzipped/111375.fb2
fb2.unzipped/1122.fb2
fb2.unzipped/113370.fb2
fb2.unzipped/113372.fb2
fb2.unzipped/114299.fb2
fb2.unzipped/114591.fb2
fb2.unzipped/115721.fb2
fb2.unzipped/115826.fb2
fb2.unzipped/119564.fb2
fb2.unzipped/122412.fb2
fb2.unzipped/123391.fb2
fb2.unzipped/124468.fb2
fb2.unzipped/124470.fb2
fb2.unzipped/124471.fb2
fb2.unzipped/125257.fb2
fb2.unzipped/131427.fb2
fb2.unzipped/131819.fb2
fb2.unzipped/132240.fb2
fb2.unzipped/132493.fb2
fb2.unzipped/132988.fb2
fb2.unzipped/133314.fb2
fb2.unzipped/133556.fb2
fb2.unzipped/134319.fb2
fb2.unzipped/13578.fb2
fb2.unzipped/137488.fb2
fb

fb2.unzipped/47228.fb2
fb2.unzipped/472356.fb2
fb2.unzipped/474349.fb2
fb2.unzipped/474364.fb2
fb2.unzipped/490942.fb2
fb2.unzipped/492702.fb2
fb2.unzipped/495691.fb2
fb2.unzipped/495809.fb2
fb2.unzipped/497370.fb2
fb2.unzipped/498584.fb2
fb2.unzipped/499588.fb2
fb2.unzipped/50901.fb2
fb2.unzipped/51355.fb2
fb2.unzipped/53075.fb2
fb2.unzipped/53076.fb2
fb2.unzipped/53078.fb2
fb2.unzipped/53364.fb2
fb2.unzipped/53564.fb2
fb2.unzipped/53580.fb2
fb2.unzipped/53607.fb2
fb2.unzipped/56643.fb2
fb2.unzipped/57678.fb2
fb2.unzipped/57681.fb2
fb2.unzipped/58521.fb2
fb2.unzipped/61787.fb2
fb2.unzipped/61821.fb2
fb2.unzipped/65796.fb2
fb2.unzipped/65797.fb2
fb2.unzipped/65864.fb2
fb2.unzipped/65888.fb2
fb2.unzipped/66372.fb2
fb2.unzipped/66373.fb2
fb2.unzipped/67536.fb2
fb2.unzipped/67550.fb2
fb2.unzipped/67903.fb2
fb2.unzipped/68317.fb2
fb2.unzipped/68319.fb2
fb2.unzipped/68344.fb2
fb2.unzipped/68345.fb2
fb2.unzipped/69159.fb2
fb2.unzipped/69167.fb2
fb2.unzipped/69300.fb2
fb2.unzipped/69716.fb2
f

In [21]:
import subprocess
import os
import time

processes = set()
max_processes = 5

os.makedirs("json", exist_ok=True)

for f in progressify_list(os.listdir('txt')):
    if (f[-3:] == "txt"):
        json_file_name = "json/" + f + ".json"
        if not os.path.isfile(json_file_name):
            processes.add(subprocess.Popen(["mystem", "--format=json", "-nigfcsd", "txt/" + f, json_file_name]))
            while len(processes) >= max_processes:
                time.sleep(0.2)
                processes.difference_update(
                    [p for p in processes if p.poll() is not None])
            print("creating", json_file_name)
        else:
            print("exists", json_file_name)
    
#Check if all the child processes were closed
for p in processes:
    if p.poll() is None:
        p.wait()


creating json/100333.fb2.txt.json
creating json/101185.fb2.txt.json
creating json/102112.fb2.txt.json
creating json/102645.fb2.txt.json
creating json/102649.fb2.txt.json
creating json/103387.fb2.txt.json
creating json/103588.fb2.txt.json
creating json/10547.fb2.txt.json
creating json/105610.fb2.txt.json
creating json/105764.fb2.txt.json
creating json/1061.fb2.txt.json
creating json/1068.fb2.txt.json
creating json/1070.fb2.txt.json
creating json/107678.fb2.txt.json
creating json/107708.fb2.txt.json
creating json/109209.fb2.txt.json
creating json/110929.fb2.txt.json
creating json/111375.fb2.txt.json
creating json/1122.fb2.txt.json
creating json/113370.fb2.txt.json
creating json/113372.fb2.txt.json
creating json/114299.fb2.txt.json
creating json/114591.fb2.txt.json
creating json/115721.fb2.txt.json
creating json/115826.fb2.txt.json
creating json/119564.fb2.txt.json
creating json/122412.fb2.txt.json
creating json/123391.fb2.txt.json
creating json/124468.fb2.txt.json
creating json/124470.fb

creating json/288557.fb2.txt.json
creating json/290867.fb2.txt.json
creating json/295292.fb2.txt.json
creating json/298923.fb2.txt.json
creating json/299188.fb2.txt.json
creating json/305713.fb2.txt.json
creating json/308939.fb2.txt.json
creating json/310583.fb2.txt.json
creating json/310689.fb2.txt.json
creating json/311032.fb2.txt.json
creating json/312454.fb2.txt.json
creating json/312635.fb2.txt.json
creating json/313626.fb2.txt.json
creating json/315588.fb2.txt.json
creating json/315732.fb2.txt.json
creating json/320065.fb2.txt.json
creating json/320174.fb2.txt.json
creating json/322391.fb2.txt.json
creating json/323998.fb2.txt.json
creating json/324990.fb2.txt.json
creating json/329591.fb2.txt.json
creating json/330813.fb2.txt.json
creating json/331966.fb2.txt.json
creating json/334920.fb2.txt.json
creating json/336019.fb2.txt.json
creating json/340699.fb2.txt.json
creating json/343914.fb2.txt.json
creating json/347944.fb2.txt.json
creating json/348600.fb2.txt.json
creating json/

In [97]:
def get_id_by_file(f):
    return f.split("/")[-1].split(".")[0]
downloaded_ids = { get_id_by_file(f) for f in os.listdir("txt") }
all_lists[(all_lists.cat_id.isnull() | ~all_lists.cat_id.isin(downloaded_ids)) & all_lists.polka_title.notnull()]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
45,45,Литературные памятники 262,Роман в восьми главах с прологом и эпилогом,Белый,Андрей,389679,1981,ru,,Петербург,NaN,NaN,NaN,NaN,NaN,NaN,Андрей Белый,Петербург,Белый Андрей
134,134,Литературные памятники 324,Роман в четырех частях,Гончаров,Иван,485060,1987,ru,Александрович,Обломов,Гончаров Иван Александрович,,Обломов,,https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D0%...,1859,Иван Гончаров,Обломов,Гончаров Иван
146,146,Литературные памятники 325,"Второе издание, дополненное",Грибоедов,Александр,397134,1987,ru,Сергеевич,Горе от ума,NaN,NaN,NaN,NaN,NaN,NaN,Александр Грибоедов,Горе от ума,Грибоедов Александр
169,169,Литературные памятники 650,,Достоевский,Федор,492140,2015,ru,Михайлович,Бедные люди,NaN,NaN,NaN,NaN,NaN,NaN,Федор Достоевский,Бедные люди,Достоевский Федор
175,175,Литературные памятники 154,,Достоевский,Федор,261797,1970,ru,Михайлович,Преступление и наказание,Достоевский Федор Михайлович,http://az.lib.ru/d/dostoewskij_f_m/,Преступление и наказание,https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D1%...,https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%...,1866,Федор Достоевский,Преступление и наказание,Достоевский Федор
305,306,,,Пелевин,Виктор,42053,,ru,Олегович,Чапаев и пустота,Пелевин Виктор Олегович,,Чапаев и пустота,,https://ru.wikipedia.org/wiki/%D0%A7%D0%B0%D0%...,1996,Виктор Пелевин,Чапаев и Пустота,Пелевин Виктор
350,351,Литературные памятники 299,"2-е изд., доп",Пушкин,Александр,276903,1984,ru,Сергеевич,Капитанская дочка,Пушкин Александр Сергеевич,,Капитанская дочка,,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...,1836,Александр Пушкин,Капитанская дочка,Пушкин Александр
403,404,Литературные памятники 352,,Сухово-Кобылин,Александр,147229,1989,ru,Васильевич,Картины прошедшего,NaN,NaN,NaN,NaN,NaN,NaN,Александр Сухово-Кобылин,Картины прошедшего,Сухово-Кобылин Александр
413,414,Анна Каренина 0|Литературные памятники 159,Роман в восьми частях,Толстой,Лев,416697,1970,ru,Николаевич,Анна Каренина,Толстой Лев николаевич,,Анна Каренина,,https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%...,1877,Лев Толстой,Анна Каренина,Толстой Лев
416,417,Детство. Отрочество. Юность 0|Литературные пам...,,Толстой,Лев,535985,1978,ru,Николаевич,Детство. Отрочество. Юность,NaN,NaN,NaN,NaN,NaN,NaN,Лев Толстой,Детство. Отрочество. Юность,Толстой Лев


In [98]:
author_downloaded_counts = for_grouping[for_grouping.cat_id.notnull() & for_grouping.cat_id.isin(downloaded_ids)].fi.value_counts()
author_downloaded_counts = pd.DataFrame(list(author_downloaded_counts.items()), columns=["f", "book_ct"])
single_authors = list(author_downloaded_counts[author_downloaded_counts.book_ct == 1].f)
for_grouping[for_grouping.fi.isin(single_authors)]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,mukhin_fio,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi
37,37,,,Ахматова,Анна,134319,2007,ru,Андреевна,Поэма без героя,NaN,NaN,NaN,NaN,NaN,NaN,Анна Ахматова,Поэма без героя,Ахматова Анна
45,45,Литературные памятники 262,Роман в восьми главах с прологом и эпилогом,Белый,Андрей,389679,1981,ru,,Петербург,NaN,NaN,NaN,NaN,NaN,NaN,Андрей Белый,Петербург,Белый Андрей
46,46,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245747,2010,ru,Романович,Голова профессора Доуэля,Беляев Александр Романович,http://lib.ru/RUFANT/BELAEW/,Голова профессора Доуэля,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D0%...,1923 - 1928,NaN,NaN,Беляев Александр
47,47,,,Беляев,Александр,409106,1994,ru,Романович,Остров погибших кораблей,Беляев Александр Романович,,Остров погибших кораблей,,https://ru.wikipedia.org/wiki/%D0%9E%D1%81%D1%...,1923 - 1928,NaN,NaN,Беляев Александр
48,48,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245745,2009,ru,Романович,Продавец воздуха,Беляев Александр Романович,,Продавец воздуха,,https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%...,ок.1929,NaN,NaN,Беляев Александр
49,49,,,Беляев,Александр,145500,2001,ru,Романович,"Человек, потерявший лицо",Беляев Александр Романович,,"Человек, потерявший лицо",,https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D0%...,1923 - 1928,NaN,NaN,Беляев Александр
50,50,Отцы основатели - русское пространство. Алекса...,,Беляев,Александр,245744,2009,ru,Романович,Человек-амфибия,Беляев Александр Романович,,Человек-амфибия,,https://ru.wikipedia.org/wiki/%D0%A7%D0%B5%D0%...,1927,NaN,NaN,Беляев Александр
56,56,Мировая классика 0,,Блок,Александр,417985,2008,ru,Александрович,Двенадцать,NaN,NaN,NaN,NaN,NaN,NaN,Александр Блок,Двенадцать,Блок Александр
65,65,Театр теней 1,,Булычев,Кир,108330,2005,ru,,Вид на битву с высоты,Булычев Кир,,Вид на битву с высоты,,,1998,NaN,NaN,Булычев Кир
66,66,Великий Гусляр 0|Классическая библиотека прикл...,сборник,Булычев,Кир,408502,2015,ru,,Перпендикулярный мир,Булычев Кир,,Перпендикулярный мир,,,1989,NaN,NaN,Булычев Кир


In [91]:
from pathlib import Path


def get_non_prose_metrics(file_path):
    lines = [l for l in Path(file_path).read_text(encoding="utf8").split("\n") if l]
    descr = all_lists[all_lists.cat_id == get_id_by_file(file_path)].iloc[0]
    return (
        descr["fi"],
        np.nan_to_num(descr["cat_title"]) or np.nan_to_num(descr["mukhin_title"]) or np.nan_to_num(descr["polka_title"]),
        lines[len(lines) // 2 : len(lines) // 2 + 10],
        np.average([len(l) for l in lines]),
        np.average([len(l.split(".")[0]) for l in lines])
    )

In [99]:
poem_ids = set()
poem_or_play_ids = set(['elka', '107708', '134319', '417985', '74281', '19794', '397134', '169185', '161263', '168370', '173154', '123391', '77040', '187978', '107678', '287218', '53075', '147229', '95985', '169133', '124471', '124470', '124468', '150566'])
for txt_path in os.listdir("txt"):
    f, title, sample, avg_line_len, avg_first_sent_len = get_non_prose_metrics("txt/" + txt_path)
    if avg_line_len < 50:
        poem_ids.add(get_id_by_file(txt_path))
        #print(str(len(poem_ids)) + ".", f, title)
        #print("\n".join(sample), "\n________\n")
    if avg_first_sent_len < 30:
        poem_or_play_ids.add(get_id_by_file(txt_path))

C:\Users\10nikov\Anaconda3\envs\myenv\lib\site-packages\numpy\lib\function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\10nikov\Anaconda3\envs\myenv\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [100]:
for_grouping[
    for_grouping.fi.isin(single_authors) & 
    for_grouping.cat_id.isin(downloaded_ids) & 
    (for_grouping.cat_id.isin(poem_ids) |
    for_grouping.cat_id.isin(poem_or_play_ids))
][["polka_fi", "polka_title"]]

,polka_fi,polka_title
37,Анна Ахматова,Поэма без героя
56,Александр Блок,Двенадцать
264,Владимир Маяковский,Облако в штанах
283,Николай Некрасов,Кому на Руси жить хорошо
404,Александр Твардовский,Василий Тёркин
459,Марина Цветаева,Поэма Горы


In [101]:
for poem_id in set(poem_ids) - set(poem_or_play_ids):
    print(poem_id, get_non_prose_metrics("txt/" + poem_id + ".fb2.txt"))

In [102]:
for_grouping["poem_or_play"] = pd.to_numeric(for_grouping.cat_id.isin(poem_ids | poem_or_play_ids) | (for_grouping.polka_title == "Зангези"))
for_grouping["not_poem_or_play"] = ~for_grouping.poem_or_play
with_poem_count = for_grouping.groupby("fi").agg({ "poem_or_play": sum, "not_poem_or_play": sum }).reset_index()

for_grouping[for_grouping.polka_title == "Облако в штанах"]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,...,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi,poem_or_play,not_poem_or_play
264,265,,,Маяковский,Владимир,161263,,ru,Владимирович,Облако в штанах,...,NaN,NaN,NaN,NaN,NaN,Владимир Маяковский,Облако в штанах,Маяковский Владимир,True,False


In [103]:
polka_authors = set(for_grouping[for_grouping.polka_title.notnull()].fi)
print(len(polka_authors))
polka_authors
with_not_enough_prose_by_polka_authors = with_poem_count[with_poem_count.fi.isin(polka_authors) & (with_poem_count.not_poem_or_play < 2) & (with_poem_count.not_poem_or_play > 0)]
with_not_enough_prose_by_polka_authors

60


,fi,poem_or_play,not_poem_or_play
102,Петров Аввакум,0.0,1.0


In [104]:
for_grouping[for_grouping.fi.isin(with_not_enough_prose_by_polka_authors.fi) & for_grouping.not_poem_or_play]

,index,cat_collection,cat_comment,cat_f,cat_i,cat_id,cat_issue_year,cat_lang,cat_o,cat_title,...,mukhin_libru,mukhin_title,mukhin_wiki1,mukhin_wiki2,mukhin_year,polka_fi,polka_title,fi,poem_or_play,not_poem_or_play
596,0,NaN,NaN,NaN,NaN,472658,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Аввакум Петров,Житие протопопа Аввакума,Петров Аввакум,False,True


In [106]:
all_lists["poem_or_play"] = all_lists.cat_id.isin(poem_ids | poem_or_play_ids) | (for_grouping.polka_title == "Зангези")
all_lists["downloaded"] = all_lists.cat_id.isin(downloaded_ids)
#all_lists[all_lists.cat_id]
all_lists.to_csv("selected_books.csv", encoding="utf-8-sig")

In [32]:
import collections

